## Calculate Diffusion Coefficient in a Folder LOCALLY.

In [ ]:
import os
import numpy as np

# ----------------- user parameters -----------------
timestep_ps     = 0.001    # MD timestep in ps
hop_threshold_A = 1.0      # Å; displacement in one frame to count as hop
dead_frames     = 3        # frames to "cool down" after each hop
jump_distance_A = 2.35     # Å; Si–Si jump distance
dump_name       = "state_dump"

# Limit how many frames to read from each big dump file.
# Set to None to read ALL frames (slow for 2.8 GB files).
max_frames      = 20000
# ---------------------------------------------------


def analyze_hops_from_dump(path, timestep_ps, hop_threshold_A, dead_frames, max_frames=None):
    """
    EXACTLY your hopping logic, generalized into a function.
    Counts hops for ALL atoms (no type selection).
    Optionally stops after max_frames frames for speed.
    """
    with open(path, "r") as f:
        frame_idx = 0
        natoms = None
        rprev_wrap = None
        rprev_unwrap = None
        last_hop_frame = None
        L = None
        first_step = None
        last_step = None
        total_hops = 0

        while True:
            if max_frames is not None and frame_idx >= max_frames:
                # stop early for speed
                print(f"  Reached max_frames={max_frames}, stopping read for this dump.")
                break

            line = f.readline()
            if not line:
                break  # EOF

            if not line.startswith("ITEM: TIMESTEP"):
                raise RuntimeError(f"Expected 'ITEM: TIMESTEP', got: {line.strip()}")

            step = int(f.readline().strip())
            if first_step is None:
                first_step = step
            last_step = step

            # number of atoms
            line = f.readline().strip()
            if not line.startswith("ITEM: NUMBER OF ATOMS"):
                raise RuntimeError("Expected 'ITEM: NUMBER OF ATOMS'")
            n = int(f.readline().strip())
            if natoms is None:
                natoms = n
            elif n != natoms:
                raise RuntimeError(f"Inconsistent natoms: previously {natoms}, now {n}")

            # box bounds
            line = f.readline().strip()
            if not line.startswith("ITEM: BOX BOUNDS"):
                raise RuntimeError("Expected 'ITEM: BOX BOUNDS'")
            bounds = []
            for _ in range(3):
                parts = f.readline().split()
                bounds.append((float(parts[0]), float(parts[1])))
            Lx, Ly, Lz = [hi - lo for lo, hi in bounds]
            if L is None:
                L = np.array([Lx, Ly, Lz], dtype=float)

            # atoms header
            header = f.readline().split()
            if header[0] != "ITEM:" or header[1] != "ATOMS":
                raise RuntimeError("Expected 'ITEM: ATOMS' header")
            labels = header[2:]

            try:
                ix = labels.index("x")
                iy = labels.index("y")
                iz = labels.index("z")
            except ValueError:
                raise RuntimeError("Dump must contain x y z columns")

            id_idx = labels.index("id") if "id" in labels else None

            coords_wrapped = np.zeros((natoms, 3), float)

            # read coordinates for this frame
            for i in range(n):
                cols = f.readline().split()
                atom_id = int(cols[id_idx]) - 1 if id_idx is not None else i
                coords_wrapped[atom_id, 0] = float(cols[ix])
                coords_wrapped[atom_id, 1] = float(cols[iy])
                coords_wrapped[atom_id, 2] = float(cols[iz])

            if frame_idx == 0:
                # initialize
                rprev_wrap   = coords_wrapped.copy()
                rprev_unwrap = coords_wrapped.copy()
                last_hop_frame = -np.ones(natoms, dtype=int)
            else:
                # unwrap using MIC
                delta = coords_wrapped - rprev_wrap
                for dim, Ldim in enumerate(L):
                    big_pos = delta[:, dim] >  0.5 * Ldim
                    big_neg = delta[:, dim] < -0.5 * Ldim
                    delta[big_pos, dim] -= Ldim
                    delta[big_neg, dim] += Ldim
                rcurr_unwrap = rprev_unwrap + delta

                # displacement in this frame
                dr = rcurr_unwrap - rprev_unwrap
                dr_mag = np.linalg.norm(dr, axis=1)

                # hopping criterion
                candidates = dr_mag >= hop_threshold_A
                can_hop = (frame_idx - last_hop_frame) > dead_frames
                new_hops = candidates & can_hop

                nhops_this_frame = int(np.count_nonzero(new_hops))
                total_hops += nhops_this_frame
                last_hop_frame[new_hops] = frame_idx

                rprev_wrap   = coords_wrapped
                rprev_unwrap = rcurr_unwrap

            frame_idx += 1

            # simple progress print every 1000 frames
            if frame_idx % 1000 == 0:
                print(f"  ...read {frame_idx} frames so far")

    if first_step is None or last_step is None:
        raise RuntimeError("No frames read from dump")

    total_time_ps = (last_step - first_step) * timestep_ps
    return natoms, total_hops, total_time_ps


def compute_D_from_hops(natoms, total_hops, total_time_ps, jump_distance_A):
    """
    Turn hop count into D, following your formula.
    """
    if total_hops == 0 or total_time_ps <= 0.0 or natoms <= 0:
        return 0.0, 0.0, 0.0

    a2_A2 = jump_distance_A ** 2
    D_A2_per_ps = a2_A2 * total_hops / (6.0 * natoms * total_time_ps)

    T_s = total_time_ps * 1e-12          # ps -> s
    a_m = jump_distance_A * 1e-10        # Å -> m
    D_m2_per_s = a_m ** 2 * total_hops / (6.0 * natoms * T_s)
    D_cm2_per_s = D_m2_per_s * 1e4

    return D_A2_per_ps, D_m2_per_s, D_cm2_per_s


def main():
    root_dir = os.path.dirname(os.path.abspath(__file__))
    print(f"Root folder: {root_dir}")

    condition_dirs = []
    for name in sorted(os.listdir(root_dir)):
        path = os.path.join(root_dir, name)
        dump_path = os.path.join(path, dump_name)
        if os.path.isdir(path) and os.path.isfile(dump_path):
            condition_dirs.append((name, dump_path))

    if not condition_dirs:
        print("No subfolders with a 'state_dump' file were found.")
        return

    print("Found conditions:")
    for label, _ in condition_dirs:
        print("  -", label)

    out_lines = []
    header = "label natoms total_hops total_time_ps D_A2_per_ps D_m2_per_s D_cm2_per_s"
    out_lines.append(header)

    for label, dump_path in condition_dirs:
        print(f"\nProcessing {label} ...")
        natoms, total_hops, total_time_ps = analyze_hops_from_dump(
            dump_path,
            timestep_ps=timestep_ps,
            hop_threshold_A=hop_threshold_A,
            dead_frames=dead_frames,
            max_frames=max_frames,
        )

        D_A2_ps, D_m2_s, D_cm2_s = compute_D_from_hops(
            natoms,
            total_hops,
            total_time_ps,
            jump_distance_A,
        )

        line = (
            f"{label} {natoms} {total_hops} "
            f"{total_time_ps:.4f} {D_A2_ps:.6e} {D_m2_s:.6e} {D_cm2_s:.6e}"
        )
        out_lines.append(line)
        print("  " + line)

    out_path = os.path.join(root_dir, "D_results_hop_multi.txt")
    with open(out_path, "w") as f:
        f.write("\n".join(out_lines))

    print(f"\nSaved hop-based diffusion results to:\n{out_path}")


if __name__ == "__main__":
    main()
